# Run clip-retrieval back with fondant-ai/datacomp-small-clip index

### Create virtual environment

In [1]:
!python3 -m venv .env
!source .env/bin/activate

## Download index and metadata

### Install requirements

In [2]:
!pip install dask[dataframe] huggingface_hub

zsh:1: no matches found: dask[dataframe]


### Create the index folder

In [3]:
!mkdir datacomp_small

### Download the index

In [5]:
!wget -O datacomp_small/image.index "https://huggingface.co/datasets/fondant-ai/datacomp-small-clip/resolve/main/faiss?download=true" -q --show-progress

datacomp_small/imag 100%[===================>]   2.33G  17.0MB/s    in 2m 4s   


### Download the metadata

In [6]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

ddf = dd.read_parquet("hf://datasets/fondant-ai/datacomp-small-clip/id_mapping")
ddf = ddf.rename(columns={"image_path": "url"})
ddf = ddf.repartition(npartitions=1)

with ProgressBar():
    ddf.to_parquet("datacomp_small/metadata")

[########################################] | 100% Completed | 39.77 s


## Run clip-retrieval backend

### Install requirements

In [7]:
!pip install clip-retrieval

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached clip_retrieval-2.44.0-py3-none-any.whl (343 kB)
  Using cached wandb-0.16.5-py3-none-any.whl (2.2 MB)
  Using cached flask-3.0.2-py3-none-any.whl (101 kB)
  Using cached autofaiss-2.17.0-py3-none-any.whl (70 kB)
  Using cached multilingual_clip-1.0.10-py3-none-any.whl (20 kB)
  Using cached Flask_Cors-4.0.0-py2.py3-none-any.whl (14 kB)
  Using cached img2dataset-1.45.0-py3-none-any.whl (42 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00
  Using cached all_clip-1.2.0-py3-none-any.whl (10 kB)
  Using cached Flask_RESTful-0.3.10-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.1 MB/s 

In [8]:
%%writefile indices.json
{
    "fondant_datacomp_small": {
        "indice_folder": "datacomp_small",
        "columns_to_return": ["url"],
        "clip_model": "open_clip:ViT-B-32/laion2b_s34b_b79k",
        "enable_mclip_option": false,
        "provide_aesthetic_embeddings": false
    }
}

Writing indices.json


In [11]:
!clip-retrieval back --port 1234 --indices-paths indices.json --clip_model open_clip:ViT-B-32/laion2b_s34b_b79k

starting boot of clip back
warming up with batch size 1 on cpu
done warming up in 2.1474368572235107s
indices loaded
 * Serving Flask app 'clip_retrieval.clip_back'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:1234
 * Running on http://10.137.96.167:1234
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Mar/2024 12:37:06] "POST /knn-service HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Mar/2024 12:37:06] "POST /metadata HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Mar/2024 12:37:11] "GET /?back=http://localhost:1234/&index=fondant_datacomp_small&useMclip=false&query=rick+and+morty HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Mar/2024 12:37:11] "GET /vendor/webcomponents-loader.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Mar/2024 12:37:11] "GET /vendor/custom-elements-es5-adapter.js